In [1]:
import requests
from bs4 import BeautifulSoup
from re import match
import json
import time
import folium

In [2]:
url = 'https://www.bhc.co.kr/location/search.asp'
response = requests.get(url)
soup = BeautifulSoup(response.text)

In [14]:
val_list = []

for option in soup.select('option') :
    if match('[가-힣]', option['value']) :
        val_list.append(option['value'])

val_list

['강원',
 '경기',
 '경남',
 '경북',
 '광주',
 '대구',
 '대전',
 '부산',
 '서울',
 '세종',
 '울산',
 '인천',
 '전남',
 '전북',
 '제주',
 '충남',
 '충북']

In [144]:
url = 'https://www.bhc.co.kr/location/search_sub.asp'
params = {
    'cPage':1,
    'search_sido':val_list[4]
}
response = requests.get(url, params=params)
soup = BeautifulSoup(response.text)

In [200]:
name_list = []

for p in soup.select('p') :
    if p.a != None :
        if p.a.text[-1] == ' ' :
            name_list.append(p.a.text[:-1])
        else :
            name_list.append(p.a.text)
    elif p.div != None:
        name_list.append(p.div.text)

name_list

['충북단양점',
 '충북대점',
 '충북보은점',
 '충북속리산점',
 '충북영동점',
 '충북옥천점',
 '충북음성점',
 '충북증평점',
 '충북진천점',
 '충북혁신도시점']

In [114]:
add_list = []
tel_list = []

for p in soup.select('p.mt10') :
    address = p.text.split('\n')[1][5:]
    telephone = p.text.split('\n')[2][7:]
    
    if (address[-1] == ' ') :
        add_list.append(address[:-1])
    else :
        add_list.append(address)

    if telephone != '' :
        if (telephone[-5] == '-') :
            tel_list.append(telephone)
        else :
            if telephone[:2] == '02' :
                telephone = telephone[:2] + '-' + telephone[2:-4] + '-' + telephone[:4]
                tel_list.append(telephone)
            else : 
                telephone = telephone[:3] + '-' + telephone[3:-4] + '-' + telephone[:4]
                tel_list.append(telephone)
    else :
        tel_list.append('None')

add_list, tel_list

(['광주 북구 군왕로 176 (두암동)',
  '광주 동구 동계천로 14-1 (대인동)',
  '광주 동구 대의동 81-5',
  '광주 서구 풍금로171번길 7 (금호동, 금수회관), 1층',
  '광주 동구 육판서길 17 (내남동, 내남지구2차진아리채) 상가동 2층 205호',
  '광주 남구 효덕로 196 (노대동)',
  '광주 서구 월산로 221 (농성동)',
  '광주 남구 고싸움로 142 (지석동)',
  '광주 동구 동계로 15 (동명동) 1층(동명동)',
  '광주 서구 동천동 525'],
 ['062-462-1577',
  '062-223-7775',
  '062-223-2727',
  '062-653-0092',
  '010-7172-6695',
  '062-676-8292',
  '062-352-2552',
  '062-373-6032',
  '062-222-0622',
  '062-515-9982'])

In [201]:
# 크롤링 전체코드
name_list = []
add_list = []
tel_list = []

for val in val_list :
    page = 0
    
    while True :
        time.sleep(0.5)
        page += 1
        
        url = 'https://www.bhc.co.kr/location/search_sub.asp'
        params = {
            'cPage':page,
            'search_sido':val
        }
        response = requests.get(url, params=params)
        soup = BeautifulSoup(response.text)
        
        if soup.select_one('tbody').tr != None :
            for p in soup.select('p') :
                if p.a != None :
                    if p.a.text[-1] == ' ' :
                        name_list.append(p.a.text[:-1])
                    else :
                        name_list.append(p.a.text)
                elif p.div != None:
                    name_list.append(p.div.text)
    
            for p in soup.select('p.mt10') :
                address = p.text.split('\n')[1][5:]
                telephone = p.text.split('\n')[2][7:]
                
                if (address[-1] == ' ') :
                    add_list.append(address[:-1])
                else :
                    add_list.append(address)

                if telephone != '' :
                    if telephone[-5] == '-' :
                        tel_list.append(telephone)
                    else :
                        if telephone[:2] == '02' :
                            telephone = telephone[:2] + '-' + telephone[2:-4] + '-' + telephone[:4]
                            tel_list.append(telephone)
                        else : 
                            telephone = telephone[:3] + '-' + telephone[3:-4] + '-' + telephone[:4]
                            tel_list.append(telephone)
                else :
                    tel_list.append('None')
        else :
            break

In [254]:
# 좌표
x_list = []
y_list = []

for i in range(len(add_list)) :
    new_add = add_list[i].split('(')[0]
    
    if new_add[-1] == ' ' :
        new_add = new_add[:-1]
    
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query':new_add
    }
    headers = {
        "Authorization":"KakaoAK REST_API_KEY"
    }
    response = requests.get(url, params=params, headers=headers)
    json_data = response.json()

    if response.json()['documents'] != [] :
        address = json_data['documents'][0]['address']
        x_list.append(address['x'])
        y_list.append(address['y'])
    else :
        x_list.append('None')
        y_list.append('None')

    time.sleep(0.5)

In [306]:
# 지도에 없는 점포 제외
rest_list = []

for i in range(len(name_list)) :
    if x_list[i] != 'None' :
        rest = {
            'name':name_list[i],
            'address':add_list[i],
            'telephone':tel_list[i],
            'x':x_list[i],
            'y':y_list[i]
        }
        rest_list.append(rest)

In [311]:
map = folium.Map(('36.3547170088274', '127.765271596878'), zoom_start=6) # 대한민국 정중앙

for rest in rest_list :
    name = rest['name']
    address = rest['address']
    telephone = rest['telephone']
    x = rest['x']
    y = rest['y']
    
    location = (y, x)
    text = f'{name}<br>{address}<br>{telephone}<br>'
    popup = folium.Popup(text, min_width=50, max_width=200)
    
    folium.Marker(
        location, 
        popup, 
        icon=folium.Icon(color='pink', icon='glyphicon-home')
    ).add_to(map)

In [312]:
map